# Introducción a la simulación

Para ***simular*** procesos de ingeniería hacemos uso de ***modelos***. Es importante enfatizar que un modelo es una ***representación*** de la realidad, no es ***la*** realidad.

Hay diferentes formas de clasificar los modelos, por ejemplo, según cómo se establece la relación de correspondencia:

* Icónicos (mediante propiedades morfológicas)
* Análogos (convenciones que codifican propiedades de la realidad)
* Simbólicos (codificación matemática)

O, por su utilidad o uso:

* Descriptivos
* Predictivos
* Prescriptivos

# Modelos aleatorios

Cuando en los procesos físicos a modelar interviene el azar, para la elaboración de modelos se utilizan generadores de números aleatorios.

## Ejemplo: Tirar un dado.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Inicializar el generador de números aleatorios
np.random.seed()
# Generar el número aleatorio
dado = np.random.randint(1, 7)
print(dado)

In [ ]:
# Tirar el dado varias veces para verificar la distribución
dados = []
frecuencias = [0, 0, 0, 0, 0, 0]
for _ in range(24):
    dado = np.random.randint(1, 7)
    dados.append(dado)
    frecuencias[dado-1] += 1
print(dados)
print(frecuencias)

In [ ]:
# Generar un histograma de los resultados
plt.hist(dados, align="left", bins=6, range=(1,7))
plt.show()

¿Cómo se comportan las frecuencias conforme se incrementa el número de tiradas?

In [ ]:
dados = []
tiradas = 10
#x = np.arange(1, tiradas+1)
frec_rel = []
frecuencias = np.array([0, 0, 0, 0, 0, 0])

for i in range(1, tiradas+1):
    dado = np.random.randint(1, 7)
    dados.append(dado)
    frecuencias[dado-1] += 1
    frec_rel.append(frecuencias / i)

print(dados)
print(frecuencias)
print(frec_rel)

In [ ]:
plt.plot(frec_rel, label=[1,2,3,4,5,6])
plt.legend()
plt.show()

## Ejemplo: El turista perdido (¿borracho?)

Tenemos un turista perdido en una ciudad. El turista se encuentra inicialmente en el centro de la ciudad. Selecciona aleatoriamente una dirección (norte, sur, este u oeste) y avanza una cuadra en esa dirección. Al llegar a la siguiente esquina, repite el proceso.



¿Qué tan lejos llegará del centro después de recorrer 20 cuadras?

In [ ]:
def mover_turista(x0, y0):
    # Dirección aleatoria
    delta_x = 0
    delta_y = 0
    # Obtener un número aleatorio en el rango [0, 1)
    direccion = np.random.random()
    # Dividir resultado en cuatro porciones
    if direccion < 0.25:
        delta_x = 1
    elif direccion < 0.5:
        delta_y = 1
    elif direccion < 0.75:
        delta_x = -1
    else:
        delta_y = -1
    return (x0 + delta_x, y0 + delta_y)


# Posición inicial
x = 0
y = 0
cuadras = 20
for _ in range(cuadras):
    x, y = mover_turista(x, y)
# Distancia final (cuántas cuadras se alejó del centro)
distancia = abs(x) + abs(y)
print(distancia)

¿Cuál es la distribución de probabilidad?

In [ ]:
def movimiento(num_movs, num_sims=1):
    """
    Genera simulaciones del movimiento de un "turista perdido",
    inicia en el origen (0,0) y se mueve una unidad en una dirección
    al azar (arriba, abajo, izquierda o derecha), al llegar a la siguiente
    esquina, vuelve a elegir una dirección al azar y avanza otra unidad,
    y así sucesivamente.
    
    num_movs: el número de movimientos.
    num_sims: el número de simulaciones.

    La función regresa una lista de tuplas que contiene las coordenadas finales
    para cada simulación.
    """
    # Inicializar lista de posiciones finales
    posiciones_finales = []
    # Generar los números aleatorios que se van a necesitar
    direccion = np.random.random(size=(num_movs, num_sims))
    # Inicia ciclo de simulaciones
    for i in range(num_sims):
        # Posición inicial (0, 0)
        x = y = 0
        # Inicia ciclo de movimientos
        for j in range(num_movs):
            if direccion[j, i] < 0.25:
                x += 1
            elif direccion[j, i] < 0.5:
                y += 1
            elif direccion[j, i] < 0.75:
                x -= 1
            else:
                y -= 1
        # Registrar posición final
        posiciones_finales.append((x, y))
    return posiciones_finales


In [ ]:
# Simular un movimiento de 20 cuadras 1000 veces
destinos = movimiento(20, 1000)
# Calcular las distancias
distancias = [abs(xy[0]) + abs(xy[1]) for xy in destinos]
dist_max = max(distancias)
# Obtener histograma
plt.hist(distancias, align="left", bins=range(dist_max+2))
plt.show()

¿Cuál es la probabilidad de que el turista se aleje del centro al menos cinco cuadras después de diez movimientos?

Podríamos correr la simulación muchas veces y calcular la proporción de veces en que queda a cinco o más cuadras del centro.

In [ ]:
def probabilidad(num_movs, cuadras, num_sims=1000):
    """
    Calcula la probabilidad de que el turista se encuentre al menos a un número
    dado de cuadras del centro después de un número dado de movimientos.
    """
    # Calcular posiciones finales
    pos_finales = movimiento(num_movs, num_sims)
    # Calcular las distancias
    distancias = [abs(xy[0]) + abs(xy[1]) for xy in pos_finales]
    # Verificar qué distancias son iguales o mayores a las cuadras indicadas
    llego = [dist for dist in distancias if dist >= cuadras]
    # La probabilidad es la proporción de resultados favorables
    prob = len(llego) / len(distancias)
    return prob


In [ ]:
probabilidad(10, 5)

Llenar la siguiente tabla de probabilidades

| Número de movimientos | Distancia mínima del centro | Probabilidad calculada |
|:---------------------:|:---------------------------:|:----------------------:|
| 5                     |  1 cuadra                   |                        |
| 10                    |  5 cuadras                  |                        |
| 20                    |  5 cuadras                  |                        |
| 50                    | 20 cuadras                  |                        |

¿Cómo se comporta la distancia conforme progresa el experimento?

In [ ]:
def ruta(num_movs):
    """
    Calcula la ruta del turista.
    Regresa una lista de coordenadas (x, y) que describe la ruta que ha seguido el turista.
    """
    # Posición inicial
    x = y = 0
    ruta_xy = [(x, y)]
    # Recorrer ruta
    for _ in range(num_movs):
        x, y = mover_turista(x, y)
        ruta_xy.append((x, y))
    # Regresar ruta
    return ruta_xy


In [ ]:
def distancia(par_xy):
    x, y = par_xy
    return abs(x) + abs(y)

paseo = ruta(50)
distancias = [distancia(pos) for pos in paseo]
plt.plot(distancias)
plt.show()